In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [2]:
import os
import pickle
import numpy as np
from tqdm.notebook import tqdm

from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add

In [ ]:
# BASE_DIR = '/content/drive/MyDrive/Colab Notebooks'
# WORKING_DIR = '/content/drive/MyDrive/Colab Notebooks'
BASE_DIR = '/Desktop/image_caption'
WORKING_DIR = ''

In [ ]:
# load vgg16 model
# model = VGG16()
# # restructure the model
# model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

In [ ]:
# # extract features from image
# features = {}
# directory = os.path.join(BASE_DIR, 'Images')

# for img_name in tqdm(os.listdir(directory),desc='feature are extracting'):
#     # load the image from file
#     img_path = directory + '/' + img_name
#     try:
#       image = load_img(img_path, target_size=(224, 224))
#     except UnidentifiedImageError as e:
#       continue
#     # convert image pixels to numpy array
#     image = img_to_array(image)
#     # reshape data for model
#     image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
#     # preprocess image for vgg
#     image = preprocess_input(image)
#     # extract features
#     feature = model.predict(image, verbose=0)
#     # get image ID
#     image_id = img_name.split('.')[0]
#     # store feature
#     features[image_id] = feature


In [ ]:
# pickle.dump(features, open(os.path.join(WORKING_DIR, 'features.pkl'), 'wb'))

In [13]:
# load features from pickle
with open(os.path.join(WORKING_DIR, 'features.pkl'), 'rb') as f:
    features = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '\\Desktop\\image_caption\\features.pkl'

In [5]:
with open(os.path.join(BASE_DIR, 'captions.txt'), 'r') as f:
    next(f)
    captions_doc = f.read()

In [6]:
# create mapping of image to captions
mapping = {}
# process lines
for line in tqdm(captions_doc.split('\n')):
    # split the line by comma(,)
    tokens = line.split(',')
    if len(line) < 2:
        continue
    image_id, caption = tokens[0], tokens[1:]
    # remove extension from image ID
    image_id = image_id.split('.')[0]
    # convert caption list to string
    caption = " ".join(caption)
    # create list if needed
    if image_id not in mapping:
        mapping[image_id] = []
    # store the caption
    mapping[image_id].append(caption)

  0%|          | 0/40456 [00:00<?, ?it/s]

In [ ]:
# len(mapping)

In [7]:
def clean(mapping):
    for key, captions in mapping.items():
        for i in range(len(captions)):
            # take one caption at a time
            caption = captions[i]
            # preprocessing steps
            # convert to lowercase
            caption = caption.lower()
            # delete digits, special chars, etc.,
            caption = caption.replace('[^A-Za-z]', '')
            # delete additional spaces
            caption = caption.replace('\s+', ' ')
            # add start and end tags to the caption
            caption = 'startseq ' + " ".join([word for word in caption.split() if len(word)>1]) + ' endseq'
            captions[i] = caption

In [8]:
# before preprocess of text
mapping['1000268201_693b08cb0e']

['A child in a pink dress is climbing up a set of stairs in an entry way .',
 'A girl going into a wooden building .',
 'A little girl climbing into a wooden playhouse .',
 'A little girl climbing the stairs to her playhouse .',
 'A little girl in a pink dress going into a wooden cabin .']

In [9]:
# preprocess the text
clean(mapping)

In [10]:
# after preprocess of text
mapping['1000268201_693b08cb0e']

['startseq child in pink dress is climbing up set of stairs in an entry way endseq',
 'startseq girl going into wooden building endseq',
 'startseq little girl climbing into wooden playhouse endseq',
 'startseq little girl climbing the stairs to her playhouse endseq',
 'startseq little girl in pink dress going into wooden cabin endseq']

In [ ]:
# all_captions = []
# for key in mapping:
#     for caption in mapping[key]:
#         all_captions.append(caption)

In [ ]:
# Save the tokenizer and max length for later use during prediction
# with open(os.path.join(WORKING_DIR, 'tokenizer.pkl'), 'wb') as token_file:
#     pickle.dump(tokenizer, token_file)

In [11]:
with open(os.path.join(WORKING_DIR, 'tokenizer.pkl'), 'rb') as token_file:
        tokenizer = pickle.load(token_file)

# vocab_size = len(tokenizer.word_index) + 1

In [ ]:
# tokenize the text
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(all_captions)
# vocab_size = len(tokenizer.word_index) + 1

In [ ]:
# Save the tokenizer and max length for later use during prediction
# with open(os.path.join(WORKING_DIR, 'tokenizer.pkl'), 'wb') as token_file:
#     pickle.dump(tokenizer, token_file)

In [ ]:
# with open(os.path.join(WORKING_DIR, 'tokenizer.pkl'), 'rb') as token_file:
#         tokenizer = pickle.load(token_file)

In [ ]:
# max_length_training = max(len(caption.split()) for caption in all_captions)
# max_length_training

37

In [ ]:
# # # get maximum length of the caption available
# max_length = max(len(caption.split()) for caption in all_captions)
# max_length

35

In [ ]:
# with open(os.path.join(WORKING_DIR, 'max_length.pkl'), 'wb') as maxlen_file:
#     pickle.dump(max_length, maxlen_file)

In [12]:
with open(os.path.join(WORKING_DIR, 'max_length.pkl'), 'rb') as maxlen_file:
        max_length = pickle.load(maxlen_file)

In [ ]:
# image_ids = list(mapping.keys())
# split = int(len(image_ids) * 0.90)
# train = image_ids[:split]
# test = image_ids[split:]

In [ ]:
# startseq girl going into wooden building endseq
#        X                   y
# startseq                   girl
# startseq girl              going
# startseq girl going        into
# ...........
# startseq girl going into wooden building      endseq

In [ ]:
# # create data generator to get data in batch (avoids session crash)
# def data_generator(data_keys, mapping, features, tokenizer, max_length, vocab_size, batch_size):
#     # loop over images
#     X1, X2, y = list(), list(), list()
#     n = 0
#     while 1:
#         for key in data_keys:
#             n += 1
#             captions = mapping[key]
#             # process each caption
#             for caption in captions:
#                 # encode the sequence
#                 seq = tokenizer.texts_to_sequences([caption])[0]
#                 # split the sequence into X, y pairs
#                 for i in range(1, len(seq)):
#                     # split into input and output pairs
#                     in_seq, out_seq = seq[:i], seq[i]
#                     # pad input sequence
#                     in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
#                     # encode output sequence
#                     out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]

#                     # store the sequences
#                     X1.append(features[key][0])
#                     X2.append(in_seq)
#                     y.append(out_seq)
#             if n == batch_size:
#                 X1, X2, y = np.array(X1), np.array(X2), np.array(y)
#                 yield [X1, X2], y
#                 X1, X2, y = list(), list(), list()
#                 n = 0

In [ ]:
# encoder model
# image feature layers
# inputs1 = Input(shape=(4096,))
# fe1 = Dropout(0.4)(inputs1)
# fe2 = Dense(256, activation='relu')(fe1)
# # sequence feature layers
# inputs2 = Input(shape=(max_length,))
# se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
# se2 = Dropout(0.4)(se1)
# se3 = LSTM(256)(se2)

# # decoder model
# decoder1 = add([fe2, se3])
# decoder2 = Dense(256, activation='relu')(decoder1)
# outputs = Dense(vocab_size, activation='softmax')(decoder2)

# model = Model(inputs=[inputs1, inputs2], outputs=outputs)
# model.compile(loss='categorical_crossentropy', optimizer='adam')

# plot the model
# plot_model(model, show_shapes=True)

In [ ]:
# # train the model
# epochs = 15
# batch_size = 32
# steps = len(train) // batch_size

# for i in range(epochs):
#     # create data generator
#     generator = data_generator(train, mapping, features, tokenizer, max_length, vocab_size, batch_size)
#     # fit for one epoch
#     model.fit(generator, epochs=1, steps_per_epoch=steps, verbose=1)

In [ ]:
# model.save('/content/drive/MyDrive/Colab Notebooks/modeltrain1.h5')

In [ ]:
# from tensorflow.keras.models import load_model
# model = load_model('/content/drive/MyDrive/Colab Notebooks/modeltrain1.h5')

In [13]:
def idx_to_word(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

In [14]:
# generate caption for an image
def predict_caption(model, image, tokenizer, max_length):
    # add start tag for generation process
    in_text = 'startseq'
    # iterate over the max length of sequence
    for i in range(max_length):
        # encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        # pad the sequence
        sequence = pad_sequences([sequence], max_length)
        # predict next word
        yhat = model.predict([image, sequence], verbose=0)
        # get index with high probability
        yhat = np.argmax(yhat)
        # convert index to word
        word = idx_to_word(yhat, tokenizer)
        # stop if word not found
        if word is None:
            break
        # append word as input for generating next word
        in_text += " " + word
        # stop if we reach end tag
        if word == 'endseq':
            break

    return in_text

In [ ]:
# from nltk.translate.bleu_score import corpus_bleu
# # validate with test data
# actual, predicted = list(), list()

# for key in tqdm(test):
#     # get actual caption
#     captions = mapping[key]
#     # predict the caption for image
#     y_pred = predict_caption(model, features[key], tokenizer, max_length)
#     # split into words
#     actual_captions = [caption.split() for caption in captions]
#     y_pred = y_pred.split()
#     # append to the list
#     actual.append(actual_captions)
#     predicted.append(y_pred)

# # calcuate BLEU score
# print("BLEU-1: %f" % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
# print("BLEU-2: %f" % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))

In [ ]:
# from PIL import Image
# import matplotlib.pyplot as plt


# class CaptionGenerator:
#     def __init__(self):
#         # Initialize your class if needed
#         pass

#     def generate_caption(self, image_name):
#         # Your existing code for generating captions
#     # def generate_caption(image_name):
#         # load the image
#         # image_name = "1001773457_577c3a7d70.jpg"
#         image_id = image_name.split('.')[0]
#         img_path = os.path.join(BASE_DIR, "Images", image_name)
#         image = Image.open(img_path)
#         captions = mapping[image_id]
#         print('---------------------Actual---------------------')
#         for caption in captions:
#             print(caption)
#         # predict the caption
#         y_pred = predict_caption(model, features[image_id], tokenizer, max_length)
#         print('--------------------Predicted--------------------')
#         print(y_pred)
#         plt.imshow(image)

# # Create an instance of the CaptionGenerator class
# models = CaptionGenerator()

In [ ]:
# models.generate_caption("230016181_0c52b95304.jpg")

In [15]:
from tensorflow.keras.models import load_model
model = load_model('/content/drive/MyDrive/Colab Notebooks/modeltrain1.h5')

In [16]:
vgg_model = VGG16()
# restructure the model
vgg_model = Model(inputs=vgg_model.inputs, outputs=vgg_model.layers[-2].output)

553467096/553467096 [==============================] - 5s 0us/step


In [17]:
image_path = '/content/drive/MyDrive/Colab Notebooks/Images/10815824_2997e03d76.jpg'
# load image
image = load_img(image_path, target_size=(224, 224))
# convert image pixels to numpy array
image = img_to_array(image)
# reshape data for model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# preprocess image for vgg
image = preprocess_input(image)
# extract features
feature = vgg_model.predict(image, verbose=0)
# predict from the trained model
predict_caption(model, feature, tokenizer, max_length)

'startseq horse horse in brown sweatshirt and rider in black sweatshirt and black hat on horse endseq'